## Covid -19 Question and Answer with Bert

In [1]:
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
from transformers import BartTokenizer, BartForConditionalGeneration
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/jdk-11.0.2"
import pandas as pd
from pyserini.search import pysearch
import numpy as np
from BERT_func import BERT_SQUAD_QA
import json
import tensorflow as tf
import tensorflow_hub as hub


## Introduction

## Methodology

### Bert

## Contribution
* 1.
* 2.
* 3.

## Setup Envirnment

In [2]:
torch_device = 'cpu'
minDate = '2020/04/02'
luceneDir = 'lucene-index-covid-2020-04-03/'

In [3]:
QA_MODEL = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
QA_TOKENIZER = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
QA_MODEL.to(torch_device)
QA_MODEL.eval()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12,

## Embbeding Method

In [4]:
def embed_useT(module):
    with tf.Graph().as_default():
        sentences = tf.compat.v1.placeholder(tf.string)
        embed = hub.Module(module)
        embeddings = embed(sentences)
        session = tf.compat.v1.train.MonitoredSession()
    return lambda x: session.run(embeddings, {sentences: x})
embed_fn = embed_useT('./kaggle/working/sentence_wise_email/module/module_useT')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


## Display the result

In [5]:
workingPath = '/kaggle/working'
import pandas as pd
from IPython.core.display import display, HTML

#from summarizer import Summarizer
#summarizerModel = Summarizer()
def displayResults(hit_dictionary, answers, question, abst):
    
    question_HTML = '<div style="font-family: Times New Roman; font-size: 28px; padding-bottom:28px"><b>Query</b>: '+question+'</div>'
    #all_HTML_txt = question_HTML
    confidence = list(answers.keys())
    confidence.sort(reverse=True)
    
    confidence = list(answers.keys())
    confidence.sort(reverse=True)
    

    for c in confidence:
        if c>0 and c <= 1 and len(answers[c]['answer']) != 0:
            rowData = []
#             idx = answers[c]['idx']
#             title = hit_dictionary[idx]['title']
#             authors = hit_dictionary[idx]['authors'] + ' et al.'

            
            full_abs = answers[c]['abstract_bert']
            bert_ans = answers[c]['answer']
            #print(full_abs)
            
            
            split_abs = full_abs.split(bert_ans)
            sentance_beginning = split_abs[0][split_abs[0].rfind('.')+1:]
            #print (sentance_beginning)
            if len(split_abs) == 1:
                sentance_end_pos = len(full_abs)
                sentance_end =''
            else:
                sentance_end_pos = split_abs[1].find('. ')+1
                if sentance_end_pos == 0:
                    sentance_end = split_abs[1]
                else:
                    sentance_end = split_abs[1][:sentance_end_pos]
                
            #sentance_full = sentance_beginning + bert_ans+ sentance_end
            answers[c]['full_answer'] = sentance_beginning+bert_ans+sentance_end
            answers[c]['sentence_beginning'] = sentance_beginning
            answers[c]['sentence_end'] = sentance_end
            #answers[c]['title'] = title
            #answers[c]['doi'] = doi
        else:
            answers.pop(c)
            
    #print(list(answers.keys()))
    
    ## now rerank based on semantic similarity of the answers to the question
    cList = list(answers.keys())
    allAnswers = [answers[c]['full_answer'] for c in cList]
    #print('all:', allAnswers)
    
    messages = [question]+allAnswers
    
    encoding_matrix = embed_fn(messages)
    similarity_matrix = np.inner(encoding_matrix, encoding_matrix)
    rankings = similarity_matrix[1:,0]
    
    for i,c in enumerate(cList):
        answers[rankings[i]] = answers.pop(c)
    
    ## now form pandas dv
    confidence = list(answers.keys())
    confidence.sort(reverse=True)
    pandasData = []
    ranked_aswers = []
    for c in confidence:
        rowData=[]
        title = answers[c]['title']
        author = answers[c]['author']
        doi = None
        #idx = answers[c]['idx']
        #rowData += [idx]            
        sentance_html = '<div>' +answers[c]['sentence_beginning'] + " <font color='red'>"+answers[c]['answer']+"</font> "+answers[c]['sentence_end']+'</div>'
        #print (sentance_html)
        rowData += [title,author, sentance_html, c]
        pandasData.append(rowData)
        ranked_aswers.append(' '.join([answers[c]['full_answer']]))
    
    pdata2 = pandasData
        
    
    display(HTML(question_HTML))
    
    df = pd.DataFrame(pdata2, columns = ['Title','Authors', 'BERT-SQuAD Answer with Highlights', 'Confidence'])
    tit = '_'.join(question.split(' '))
    if abst:
        df.to_csv('./result/Abs+' + tit + '.csv')
        print('Search with only Abstract')
    else:
        df.to_csv('./result/Full+' + tit + '.csv')
        print ('Search with full paper')
        
    display(HTML(df.to_html(render_links=True, escape=False)))
    


In [6]:
def Display_all(query, keywords, abst):
    
    #search with luceneDir database by anserini
    searcher = pysearch.SimpleSearcher(luceneDir)
    hits = searcher.search(query + '. ' + keywords)
    n_hits = len(hits)
    
    #get database by ourselves
    with open('database.json', 'r') as fp:
        database = json.loads(fp.read())
        
    ID = []
    for i in range(0, n_hits):
        doc_json = json.loads(hits[i].raw)
        try:
            ID.append(doc_json['paper_id'])
        except:
            pass
        
    database_df = pd.DataFrame(database).T
    
    database_df['abs_text'] = database_df.abstract+ database_df['full-text']
    
    #match with own database
    ID_real = []
    for Id in ID:
        if Id in database:
            ID_real.append(Id)
    
    hit_dictionary = database_df.loc[ID_real].to_dict('index')
    
    QA_model = BERT_SQUAD_QA(QA_TOKENIZER, QA_MODEL)
    ans = QA_model.search_abstracts(hit_dictionary, query, abst)
    
    displayResults(hit_dictionary, ans, query, abst)
        

In [7]:
all_topics=[
    'What is known about transmission, incubation, and environmental stability?',
    'What do we know about COVID-19 risk factors?',
    'What do we know about virus genetics, origin, and evolution?',
    'What do we know about vaccines and therapeutics?',
    'What do we know about non-pharmaceutical interventions?',
    'What has been published about medical care?',
    'What do we know about diagnostics and surveillance?'
    'What has been published about information sharing and inter-sectoral collaboration?',
    'What has been published about ethical and social science considerations?'
]
topic_area = {}

#0
#What is known about transmission, incubation, and environmental stability?
question_list = []
kw_list = []
pm_kw_list = []
question_list.append("What is known about transmission, incubation, and environmental stability")
kw_list.append("2019-nCoV,SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, person to person,touch, copper, human to human,temperature, humidity, interpersonal contact, air, water,fecal, surfaces, aerisol, transmission, shedding")



topic_area['What is known about transmission, incubation, and environmental stability?'] = list(zip(question_list,kw_list))



#1
#What do we know about COVID-19 risk factors?
question_list = []
kw_list = []

question_list.append("What risk factors contribute to the severity of 2019-nCoV")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, susceptible, smoking, smoker, neonates, pregnant, socio-economic, behavioral, age, elderly, young, old, children")


topic_area['What do we know about COVID-19 risk factors?'] = list(zip(question_list,kw_list))


#2
#What do we know about virus genetics, origin, and evolution?
question_list = []
kw_list = []


question_list.append("What animal did 2019-nCoV come from")


topic_area['What do we know about virus genetics, origin, and evolution?'] = list(zip(question_list,kw_list))

#3
#What do we know about vaccines and therapeutics?
question_list = []
kw_list = []
pm_kw_list = []
question_list.append("What drugs or therapies or antiviral are being investigated and recommended")
kw_list.append("2019-nCoV,  COVID-19, coronavirus, novel coronavirus, drug, antiviral, testing, clinical trial, study")


topic_area['What do we know about vaccines and therapeutics?'] = list(zip(question_list,kw_list))


#4
#What do we know about non-pharmaceutical interventions?
question_list = []
kw_list = []
question_list.append("Which non-pharmaceutical interventions limit tramsission")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, non-pharmaceutical interventions, npi")


topic_area['What do we know about non-pharmaceutical interventions?'] = list(zip(question_list,kw_list))

#5
#What has been published about medical care?
question_list = []
kw_list = []


question_list.append("What adjunctive or supportive methods can help patients")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, adjunctive, supportive")


topic_area['What has been published about medical care?'] = list(zip(question_list,kw_list))

#6
#What do we know about diagnostics and surveillance?
question_list = []
kw_list = []
question_list.append("What diagnostic tests (tools) exist or are being developed to detect 2019-nCoV")
kw_list.append("2019-nCoV, SARS-CoV-2, COVID-19, coronavirus, novel coronavirus, diagnosis, tools, detetion")

topic_area['What do we know about diagnostics and surveillance?'] = list(zip(question_list,kw_list))



#7
#What has been published about information sharing and inter-sectoral collaboration?
question_list = []
kw_list = []

question_list.append('What collaborations are happening within the research community')
kw_list.append('inter-sectorial, international, collaboration, global, coronavirus, novel coronavirus, sharing')


topic_area['What has been published about information sharing and inter-sectoral collaboration?'] = list(zip(question_list,kw_list))


#8
#What has been published about ethical and social science considerations?
question_list = []
kw_list = []


question_list.append("What are the major ethical issues related pandemic outbreaks")
kw_list.append("ehtics, pandemic")


topic_area['What has been published about ethical and social science considerations?'] = list(zip(question_list,kw_list))




In [9]:
for i in all_topics:
    Display_all(topic_area[i][0][0], topic_area[i][0][1], abst = True)
    Display_all(topic_area[i][0][0], topic_area[i][0][1], abst = False)

 20%|██        | 1/5 [00:01<00:04,  1.03s/it]


ValueError: Input is not valid. Should be a string, a list/tuple of strings or a list/tuple of integers.

In [ ]:
topic_area[][0][1]